In [1]:
import pandas as pd
import requests
from tqdm.auto import tqdm
tqdm.pandas()

In [2]:
def next_monday(date):
    return pd.date_range(start=date, end=date + pd.offsets.Day(6), freq='W-MON')[0]

# Setting

In [3]:
test_date = pd.to_datetime('2021-06-14')

In [4]:
models_to_exclude = ['COVIDhub-ensemble', 'COVIDhub-trained_ensemble', 
                     'CU-nochange', 'CU-scenario_high', 'CU-scenario_low', 'CU-scenario_mid']

locations_to_exclude = ["60", "66", "69", "72", "74", "78"]

# DC,11,District of Columbia
# AS,60,American Samoa
# GU,66,Guam
# MP,69,Northern Mariana Islands
# PR,72,Puerto Rico
# UM,74,U.S. Minor Outlying Islands
# VI,78,Virgin Islands

In [5]:
dtype={'target': str, 'location': str, 'type': str, 'quantile': float, 'value': float}
parse_dates=['forecast_date', 'target_end_date']

# Load Files

In [6]:
url = "https://api.github.com/repos/reichlab/covid19-forecast-hub/git/trees/master?recursive=1"
r = requests.get(url)
res = r.json()

In [7]:
files = [file["path"] for file in res["tree"] if (file["path"].startswith('data-processed/') and file["path"].endswith('.csv'))]

In [8]:
files[-5:]

['data-processed/epiforecasts-ensemble1/2021-05-17-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-24-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-05-31-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-06-07-epiforecasts-ensemble1.csv',
 'data-processed/epiforecasts-ensemble1/2021-06-14-epiforecasts-ensemble1.csv']

In [9]:
df_files = pd.DataFrame({'filename':files})

df_files['model'] = df_files.filename.apply(lambda f: f.split('/')[1])

df_files['forecast_date'] = df_files.filename.apply(lambda f: f.split('/')[2][:10])
df_files.forecast_date = pd.to_datetime(df_files.forecast_date)

df_files['timezero'] = df_files.forecast_date.apply(next_monday)

df_files = df_files[~df_files.model.isin(models_to_exclude)]

In [10]:
df_files

,filename,model,forecast_date,timezero
0,data-processed/AIpert-pwllnod/2020-12-21-AIper...,AIpert-pwllnod,2020-12-21,2020-12-21
1,data-processed/AIpert-pwllnod/2020-12-28-AIper...,AIpert-pwllnod,2020-12-28,2020-12-28
2,data-processed/AIpert-pwllnod/2021-01-04-AIper...,AIpert-pwllnod,2021-01-04,2021-01-04
3,data-processed/AIpert-pwllnod/2021-01-11-AIper...,AIpert-pwllnod,2021-01-11,2021-01-11
4,data-processed/AIpert-pwllnod/2021-01-18-AIper...,AIpert-pwllnod,2021-01-18,2021-01-18
...,...,...,...,...
3670,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-17,2021-05-17
3671,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-24,2021-05-24
3672,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-31,2021-05-31
3673,data-processed/epiforecasts-ensemble1/2021-06-...,epiforecasts-ensemble1,2021-06-07,2021-06-07


### Test Data

In [11]:
VALID_TARGETS = [f"{_} wk ahead inc death" for _ in range(1, 5)] + \
                [f"{_} wk ahead cum death" for _ in range(1, 5)] + \
                [f"{_} wk ahead inc case" for _ in range(1, 5)]

In [12]:
df_test_files = df_files[df_files.timezero == test_date]

In [13]:
df_test_files.head()

,filename,model,forecast_date,timezero
25,data-processed/AIpert-pwllnod/2021-06-14-AIper...,AIpert-pwllnod,2021-06-14,2021-06-14
74,data-processed/BPagano-RtDriven/2021-06-13-BPa...,BPagano-RtDriven,2021-06-13,2021-06-14
126,data-processed/CEID-Walk/2021-06-14-CEID-Walk.csv,CEID-Walk,2021-06-14,2021-06-14
174,data-processed/CMU-TimeSeries/2021-06-14-CMU-T...,CMU-TimeSeries,2021-06-14,2021-06-14
237,data-processed/COVIDhub-baseline/2021-06-14-CO...,COVIDhub-baseline,2021-06-14,2021-06-14


In [14]:
dfs_test = []

for _, row in tqdm(df_test_files.iterrows(), total=df_test_files.shape[0]):
    #print(row['filename'])
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs_test.append(df_temp)

  0%|          | 0/49 [00:00<?, ?it/s]

In [15]:
df_test = pd.concat(dfs_test)

In [16]:
df_test.shape

(1673575, 8)

In [17]:
df_test

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-06-14,1 wk ahead cum death,2021-06-19,0.025,quantile,600794.136197,US,AIpert-pwllnod
1,2021-06-14,1 wk ahead cum death,2021-06-19,0.250,quantile,601872.560040,US,AIpert-pwllnod
2,2021-06-14,1 wk ahead cum death,2021-06-19,0.750,quantile,603207.473569,US,AIpert-pwllnod
3,2021-06-14,1 wk ahead cum death,2021-06-19,0.975,quantile,604673.181284,US,AIpert-pwllnod
4,2021-06-14,1 wk ahead cum death,2021-06-19,NaN,point,602438.412295,US,AIpert-pwllnod
...,...,...,...,...,...,...,...,...
10939,2021-06-14,4 wk ahead cum death,2021-07-10,0.850,quantile,610046.000000,US,epiforecasts-ensemble1
10940,2021-06-14,4 wk ahead cum death,2021-07-10,0.900,quantile,610626.000000,US,epiforecasts-ensemble1
10941,2021-06-14,4 wk ahead cum death,2021-07-10,0.950,quantile,611586.000000,US,epiforecasts-ensemble1
10942,2021-06-14,4 wk ahead cum death,2021-07-10,0.975,quantile,612661.000000,US,epiforecasts-ensemble1


In [18]:
# only consider US + 50 states
df_test = df_test[df_test.location.str.len() == 2]
df_test = df_test[~df_test.location.isin(locations_to_exclude)]

In [19]:
df_test.shape

(347900, 8)

In [20]:
df_test.location.nunique()

52

In [21]:
# ensure that for all targets each model provides forecasts for all locations
df_test = df_test[df_test.groupby(['target', 'model'])['location'].transform('nunique') == 52]

In [22]:
df_test = df_test[df_test.type == 'quantile']

In [23]:
df_test['no_quantiles'] = df_test.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df_test['no_quantiles'] = df_test.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [24]:
df_test = df_test[(df_test.no_quantiles == 23) | 
        (df_test.target.str.contains('inc case') & (df_test.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [25]:
df_test.shape

(255840, 8)

In [26]:
# dict of the models available for each target
available_models = dict(df_test.groupby(['target'])['model'].unique())

In [27]:
available_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'IHME-CurveFit',
        'JHUAPL-Bucky', 'JHU_CSSE-DECOM', 'JHU_IDD-CovidSP', 'Karlen-pypm',
        'LANL-GrowthRate', 'MIT_CritData-GBCF', 'Microsoft-DeepSTIA',
        'OliverWyman-Navigator', 'PSI-DRAFT', 'RobertWalraven-ESG',
        'SteveMcConnell-CovidComplete', 'UA-EpiCovDA', 'UCLA-SuEIR',
        'UMass-MechBayes', 'USC-SI_kJalpha', 'epiforecasts-ensemble1'],
       dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'MUNI-ARIMA',
        'Microsoft-DeepSTIA', 'RobertWalraven-ESG', 'UCLA-SuEIR',
        'USC-SI_kJalpha', 'UVA-Ensemble'], dtype=object),
 '1 wk ahead inc death': array

## Training Data

d - 4 weeks - (horizon - 1) --> end: d - horizon

In [28]:
lower_bound = test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(4 - 1))

In [29]:
lower_bound

Timestamp('2021-04-26 00:00:00')

In [30]:
df_files = df_files[(df_files.timezero >= lower_bound) & (df_files.timezero < test_date)].copy()

In [31]:
# assigns to each horizon the corresponding training forecast dates for the test date
h_dict = {}
for h in range(1, 5):
    h_dict[h] = [test_date - pd.Timedelta(weeks=4) - pd.Timedelta(weeks=(h - 1)), 
                 test_date - pd.Timedelta(weeks=h)]

In [32]:
h_dict

{1: [Timestamp('2021-05-17 00:00:00'), Timestamp('2021-06-07 00:00:00')],
 2: [Timestamp('2021-05-10 00:00:00'), Timestamp('2021-05-31 00:00:00')],
 3: [Timestamp('2021-05-03 00:00:00'), Timestamp('2021-05-24 00:00:00')],
 4: [Timestamp('2021-04-26 00:00:00'), Timestamp('2021-05-17 00:00:00')]}

In [33]:
def relevant_horizons(d):
    hs = []
    for h in range(1, 5):
        if((d >= h_dict[h][0]) & (d <= h_dict[h][1])):
            hs.append(h)
    return hs

In [34]:
df_files['horizons'] = df_files.timezero.apply(relevant_horizons)

In [35]:
df_files

,filename,model,forecast_date,timezero,horizons
18,data-processed/AIpert-pwllnod/2021-04-26-AIper...,AIpert-pwllnod,2021-04-26,2021-04-26,[4]
19,data-processed/AIpert-pwllnod/2021-05-03-AIper...,AIpert-pwllnod,2021-05-03,2021-05-03,"[3, 4]"
20,data-processed/AIpert-pwllnod/2021-05-10-AIper...,AIpert-pwllnod,2021-05-10,2021-05-10,"[2, 3, 4]"
21,data-processed/AIpert-pwllnod/2021-05-17-AIper...,AIpert-pwllnod,2021-05-17,2021-05-17,"[1, 2, 3, 4]"
22,data-processed/AIpert-pwllnod/2021-05-24-AIper...,AIpert-pwllnod,2021-05-24,2021-05-24,"[1, 2, 3]"
...,...,...,...,...,...
3669,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-10,2021-05-10,"[2, 3, 4]"
3670,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-17,2021-05-17,"[1, 2, 3, 4]"
3671,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-24,2021-05-24,"[1, 2, 3]"
3672,data-processed/epiforecasts-ensemble1/2021-05-...,epiforecasts-ensemble1,2021-05-31,2021-05-31,"[1, 2]"


In [36]:
# only keep relevant training data
df_files = df_files[df_files.horizons.apply(len) > 0]

In [37]:
dfs = []
for _, row in tqdm(df_files.iterrows(), total=df_files.shape[0]):
    VALID_TARGETS = [f"{_} wk ahead inc death" for _ in row['horizons']] + \
                    [f"{_} wk ahead cum death" for _ in row['horizons']] + \
                    [f"{_} wk ahead inc case" for _ in row['horizons']]
    df_temp = pd.read_csv('https://github.com/reichlab/covid19-forecast-hub/raw/master/' + row['filename'],
                         dtype=dtype, parse_dates=parse_dates)
    df_temp = df_temp[df_temp.target.isin(VALID_TARGETS)]
    df_temp['model'] = row['model']
    dfs.append(df_temp)

  0%|          | 0/404 [00:00<?, ?it/s]

In [38]:
df = pd.concat(dfs)

In [39]:
df.shape

(9041809, 8)

# Data Cleaning

In [40]:
df = df[df.location.str.len() == 2]
df = df[~df.location.isin(locations_to_exclude)]

In [41]:
df.location.nunique() # US + 50 states + district of Columbia

52

In [42]:
df = df[df.type == 'quantile']

In [43]:
# how many forecasts for each target/model/location? should be 4 for every location
df['no_forecasts'] = df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique')

In [44]:
df = df[df.groupby(['target', 'model'])['no_forecasts'].transform('min') == 4].drop(columns='no_forecasts').reset_index(drop=True)

In [45]:
# df = df[df.groupby(['target', 'model', 'location'])['target_end_date'].transform('nunique') == 4]
# would only remove the respective location with <4 forecasts, but we want to remove the model as a whole if in one location <4

In [46]:
df = df[df.groupby(['target', 'model', 'target_end_date'])['location'].transform('nunique') == 52]

In [47]:
df['no_quantiles'] = df.groupby(['model', 'target', 'target_end_date', 'location'])['quantile'].transform('nunique')

df['no_quantiles'] = df.groupby(['target', 'model'])['no_quantiles'].transform('min')

In [48]:
df[(df.target.str.contains('inc case') & (df.no_quantiles == 7))]

,forecast_date,target,target_end_date,quantile,type,value,location,model,no_quantiles
9048,2021-04-25,4 wk ahead inc case,2021-05-22,0.025,quantile,158320.473030,US,BPagano-RtDriven,7
9049,2021-04-25,4 wk ahead inc case,2021-05-22,0.100,quantile,237995.058120,US,BPagano-RtDriven,7
9050,2021-04-25,4 wk ahead inc case,2021-05-22,0.250,quantile,310403.394940,US,BPagano-RtDriven,7
9051,2021-04-25,4 wk ahead inc case,2021-05-22,0.500,quantile,391120.425560,US,BPagano-RtDriven,7
9052,2021-04-25,4 wk ahead inc case,2021-05-22,0.750,quantile,471837.456190,US,BPagano-RtDriven,7
...,...,...,...,...,...,...,...,...,...
1121495,2021-06-07,1 wk ahead inc case,2021-06-12,0.250,quantile,41502.813436,US,UVA-Ensemble,7
1121496,2021-06-07,1 wk ahead inc case,2021-06-12,0.500,quantile,88649.783327,US,UVA-Ensemble,7
1121497,2021-06-07,1 wk ahead inc case,2021-06-12,0.750,quantile,135796.753219,US,UVA-Ensemble,7
1121498,2021-06-07,1 wk ahead inc case,2021-06-12,0.900,quantile,178230.496834,US,UVA-Ensemble,7


In [49]:
df = df[(df.no_quantiles == 23) | 
        (df.target.str.contains('inc case') & (df.no_quantiles == 7))].drop(columns='no_quantiles').reset_index(drop=True)

In [50]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    22
1 wk ahead inc case     20
1 wk ahead inc death    24
2 wk ahead cum death    22
2 wk ahead inc case     20
2 wk ahead inc death    24
3 wk ahead cum death    20
3 wk ahead inc case     17
3 wk ahead inc death    22
4 wk ahead cum death    20
4 wk ahead inc case     16
4 wk ahead inc death    22
Name: model, dtype: int64

In [51]:
# check if there are models used for training that are not available for the test date
train_models = dict(df.groupby('target').model.unique())

a = [(k, t) for k, v in train_models.items() for t in v]
b = [(k, t) for k, v in available_models.items() for t in v]

[i for i in a if i not in b]

[('1 wk ahead cum death', 'DDS-NBDS'),
 ('1 wk ahead cum death', 'KITmetricslab-select_ensemble'),
 ('1 wk ahead cum death', 'LNQ-ens1'),
 ('1 wk ahead inc case', 'KITmetricslab-select_ensemble'),
 ('1 wk ahead inc case', 'LNQ-ens1'),
 ('1 wk ahead inc case', 'UChicagoCHATTOPADHYAY-UnIT'),
 ('1 wk ahead inc death', 'DDS-NBDS'),
 ('1 wk ahead inc death', 'KITmetricslab-select_ensemble'),
 ('1 wk ahead inc death', 'LNQ-ens1'),
 ('2 wk ahead cum death', 'DDS-NBDS'),
 ('2 wk ahead cum death', 'KITmetricslab-select_ensemble'),
 ('2 wk ahead cum death', 'LNQ-ens1'),
 ('2 wk ahead inc case', 'KITmetricslab-select_ensemble'),
 ('2 wk ahead inc case', 'LNQ-ens1'),
 ('2 wk ahead inc case', 'UChicagoCHATTOPADHYAY-UnIT'),
 ('2 wk ahead inc death', 'DDS-NBDS'),
 ('2 wk ahead inc death', 'KITmetricslab-select_ensemble'),
 ('2 wk ahead inc death', 'LNQ-ens1'),
 ('3 wk ahead cum death', 'DDS-NBDS'),
 ('3 wk ahead cum death', 'KITmetricslab-select_ensemble'),
 ('3 wk ahead cum death', 'LNQ-ens1'),
 ('3

In [52]:
df = df[df.apply(lambda x: x.model in (available_models[x.target]), axis=1)]

In [53]:
df.groupby('target').model.nunique()

target
1 wk ahead cum death    19
1 wk ahead inc case     17
1 wk ahead inc death    21
2 wk ahead cum death    19
2 wk ahead inc case     17
2 wk ahead inc death    21
3 wk ahead cum death    17
3 wk ahead inc case     15
3 wk ahead inc death    19
4 wk ahead cum death    18
4 wk ahead inc case     15
4 wk ahead inc death    20
Name: model, dtype: int64

In [54]:
df.shape

(868504, 8)

In [55]:
models_by_target = pd.DataFrame([(i, k) for i,j in train_models.items() for k in j], 
                  columns=['target','model'])

In [56]:
models_by_target[models_by_target.target == '4 wk ahead cum death']

,target,model
191,4 wk ahead cum death,BPagano-RtDriven
192,4 wk ahead cum death,CEID-Walk
193,4 wk ahead cum death,COVIDhub-baseline
194,4 wk ahead cum death,CU-select
195,4 wk ahead cum death,Covid19Sim-Simulator
196,4 wk ahead cum death,CovidAnalytics-DELPHI
197,4 wk ahead cum death,DDS-NBDS
198,4 wk ahead cum death,JHUAPL-Bucky
199,4 wk ahead cum death,JHU_IDD-CovidSP
200,4 wk ahead cum death,Karlen-pypm


In [57]:
df = df[~df.model.isin(models_to_exclude)]

In [58]:
df

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-04-25,4 wk ahead inc death,2021-05-22,0.010,quantile,1449.68565,US,BPagano-RtDriven
1,2021-04-25,4 wk ahead inc death,2021-05-22,0.025,quantile,1957.20698,US,BPagano-RtDriven
2,2021-04-25,4 wk ahead inc death,2021-05-22,0.050,quantile,2410.90431,US,BPagano-RtDriven
3,2021-04-25,4 wk ahead inc death,2021-05-22,0.100,quantile,2942.16901,US,BPagano-RtDriven
4,2021-04-25,4 wk ahead inc death,2021-05-22,0.150,quantile,3302.94463,US,BPagano-RtDriven
...,...,...,...,...,...,...,...,...
986851,2021-06-07,1 wk ahead cum death,2021-06-12,0.850,quantile,600466.00000,US,epiforecasts-ensemble1
986852,2021-06-07,1 wk ahead cum death,2021-06-12,0.900,quantile,600602.00000,US,epiforecasts-ensemble1
986853,2021-06-07,1 wk ahead cum death,2021-06-12,0.950,quantile,600826.00000,US,epiforecasts-ensemble1
986854,2021-06-07,1 wk ahead cum death,2021-06-12,0.975,quantile,601027.00000,US,epiforecasts-ensemble1


In [59]:
available_train_models = dict(df.groupby(['target'])['model'].unique())

In [60]:
available_train_models

{'1 wk ahead cum death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI', 'IHME-CurveFit',
        'JHUAPL-Bucky', 'JHU_IDD-CovidSP', 'Karlen-pypm',
        'LANL-GrowthRate', 'MIT_CritData-GBCF', 'Microsoft-DeepSTIA',
        'RobertWalraven-ESG', 'SteveMcConnell-CovidComplete',
        'UA-EpiCovDA', 'UMass-MechBayes', 'USC-SI_kJalpha',
        'epiforecasts-ensemble1'], dtype=object),
 '1 wk ahead inc case': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Simulator', 'CovidAnalytics-DELPHI',
        'IEM_MED-CovidProject', 'JHUAPL-Bucky', 'JHU_CSSE-DECOM',
        'JHU_IDD-CovidSP', 'Karlen-pypm', 'LANL-GrowthRate', 'MUNI-ARIMA',
        'Microsoft-DeepSTIA', 'RobertWalraven-ESG', 'USC-SI_kJalpha',
        'UVA-Ensemble'], dtype=object),
 '1 wk ahead inc death': array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-baseline', 'CU-select',
        'Covid19Sim-Si

In [61]:
df_test.shape

(255840, 8)

In [62]:
df_test = df_test[df_test.apply(lambda x: x.model in (available_train_models[x.target]), axis=1)]

In [63]:
df_test.head()

,forecast_date,target,target_end_date,quantile,type,value,location,model
0,2021-06-13,1 wk ahead inc death,2021-06-19,0.010,quantile,1313.91621,US,BPagano-RtDriven
1,2021-06-13,1 wk ahead inc death,2021-06-19,0.025,quantile,1444.91417,US,BPagano-RtDriven
2,2021-06-13,1 wk ahead inc death,2021-06-19,0.050,quantile,1562.01943,US,BPagano-RtDriven
3,2021-06-13,1 wk ahead inc death,2021-06-19,0.100,quantile,1699.14586,US,BPagano-RtDriven
4,2021-06-13,1 wk ahead inc death,2021-06-19,0.150,quantile,1792.26681,US,BPagano-RtDriven


In [64]:
df.groupby(['target'])['model'].unique()

target
1 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
1 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
2 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
3 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead cum death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc case     [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
4 wk ahead inc death    [BPagano-RtDriven, CEID-Walk, COVIDhub-baselin...
Name: model, dtype: object

# Export

In [65]:
df.to_csv('data/df_train_{}.csv'.format(str(test_date.date())), index=False)

In [66]:
df_test.to_csv('data/df_test_{}.csv'.format(str(test_date.date())), index=False)